In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import pandas as pd
import numpy as np
import pickle
import sys
sys.path.append('/mnt/d/projects/diachronic-p2v/')
%aimport preprocessing
%aimport utils
%aimport p2v

from utils import prinT

pd.options.display.max_columns = None
pd.set_option('max_colwidth', None)

/home/lyuzhuoqi/miniconda3/envs/p2v/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
model = p2v.P2V()

data_dir:  /mnt/d/projects/diachronic-p2v/data
2025-01-05 20:18:49 -- start loading Mag_venue_info_df
2025-01-05 20:18:49 -- finish.
2025-01-05 20:18:49 -- start loading labeled_journal_info_df
2025-01-05 20:18:49 -- finish.


# Periodicals' VID and their names in MAG

In [5]:
model.MAG_venue_info_df[model.MAG_venue_info_df['OriginalVenue'].str.contains('The Royal Society B', na=False, case=False)]

,VenueID,VenueType,OriginalVenue,PublicationVolume,PublicationVolume_1950s,PublicationVolume_1960s,PublicationVolume_1970s,PublicationVolume_1980s,PublicationVolume_1990s,PublicationVolume_2000s,PublicationVolume_2010s
219,91660768,Journal,Philosophical Transactions of the Royal Society B,11598,96,204,808,1272,1675,2232,4737
864,3006207977,Journal,Proceedings of The Royal Society B: Biological Sciences,20208,753,874,1201,1486,2407,4120,7045


# MAG periodicals with Scopus labels

In [248]:
venue_name = 'Web and Social Media'
model.labeled_journal_info_df[(model.labeled_journal_info_df['OriginalVenue'].str.contains(venue_name, case=False))|
(model.labeled_journal_info_df['ScopusName'].str.contains(venue_name, case=False))]

,VenueType,OriginalVenue,ScopusName,ScopusCategory,InceptionYear
VenueID,,,,,


# look up periodicals' neighbors in different decades

In [6]:
start_year_list = ['1950', '1960', '1970', '1980', '1990', '2000', '2010']
end_year_list = ['1959', '1969', '1979', '1989', '1999', '2009', '2021']

In [11]:
def display_neighbors(venue_name, start_year, end_year, k):
    tar_wv = model.load_wv(start_year, end_year, 100, 10)
    VID = model.MAG_venue_info_df[model.MAG_venue_info_df.OriginalVenue==venue_name].VenueID.values[0]
    tup_list = tar_wv.most_similar(positive=VID, topn=k)
    for tup in tup_list: 
        print (model.MAG_venue_info_df[model.MAG_venue_info_df['VenueID']==tup[0]].OriginalVenue.values[0], tup[1])

In [12]:
venue_name = 'Philosophical Transactions of the Royal Society B'
for i in range(0, len(start_year_list)):
    try: 
        print('------------------')
        print('find neighbors for '+venue_name+' in '+start_year_list[i]+'\'s:')
        display_neighbors(venue_name, start_year_list[i], end_year_list[i], 6)
    except KeyError:
        print("This periodical was not included in the embedding in that decade.")
        pass

------------------
find neighbors for Philosophical Transactions of the Royal Society B in 1950's:
2025-01-05 20:19:57 -- start loading word vectors...
2025-01-05 20:19:57 -- word vectors loaded, and its shape is: (1645, 100)
Zoological Journal of the Linnean Society 0.6627570986747742
Transactions of the Royal Society of Edinburgh 0.6493370532989502
EG Quaternary Science Journal 0.6428252458572388
Journal of Molluscan Studies 0.6261870861053467
Madroño; a West American journal of botany 0.6048715114593506
The Transactions of The Zoological Society of London 0.5983977317810059
------------------
find neighbors for Philosophical Transactions of the Royal Society B in 1960's:
2025-01-05 20:19:57 -- start loading word vectors...
2025-01-05 20:19:57 -- word vectors loaded, and its shape is: (3116, 100)
Earth and Environmental Science Transactions of The Royal Society of Edinburgh 0.7481801509857178
Journal of Molluscan Studies 0.6915721893310547
Zoological Journal of the Linnean Society 0.

# Look up periodicals' papers (Paper_detialed_df)

In [13]:
with open('/mnt/d/projects/diachronic-p2v/data/paper_detailed.pkl', 'rb') as file:
    model.paper_detailed_df = pickle.load(file) 

In [20]:
venue_name = 'Philosophical Transactions of the Royal Society B'
start_year = '1980'
end_year = '1989'

print("total number of papers:", len(model.paper_detailed_df[start_year:end_year][(model.paper_detailed_df[start_year:end_year].OriginalVenue==venue_name)]))
model.paper_detailed_df[start_year:end_year][(model.paper_detailed_df[start_year:end_year].
                                              OriginalVenue==venue_name)].sort_values('CitationCount', ascending=False).head(20)

total number of papers: 1272


,PaperID,Doi,DocType,PaperTitle,JournalID,ConferenceSeriesID,ReferenceCount,CitationCount,OriginalVenue,VenueID
Year,,,,,,,,,,
1986-01-01,2160938187,10.1098/RSTB.1986.0056,Journal,the structure of the nervous system of the nematode caenorhabditis elegans,91660768,0,40,4592,Philosophical Transactions of the Royal Society B,91660768
1982-01-01,2142004577,10.1098/RSTB.1982.0082,Journal,specific impairments of planning,91660768,0,27,2982,Philosophical Transactions of the Royal Society B,91660768
1989-01-01,2117066938,10.1098/RSTB.1989.0106,Journal,the phylogenetic regression,91660768,0,8,1698,Philosophical Transactions of the Royal Society B,91660768
1980-01-01,2143874510,10.1098/RSTB.1980.0049,Journal,the structure of beta lactamases,91660768,0,16,1334,Philosophical Transactions of the Royal Society B,91660768
1987-01-01,2037428143,10.1098/RSTB.1987.0030,Journal,ecological morphology and flight in bats mammalia chiroptera wing adaptations flight performance foraging strategy and echolocation,91660768,0,172,1332,Philosophical Transactions of the Royal Society B,91660768
1982-01-01,2106235350,10.1098/RSTB.1982.0074,Journal,a memory system in the monkey,91660768,0,23,996,Philosophical Transactions of the Royal Society B,91660768
1981-01-01,1978948864,10.1098/RSTB.1981.0005,Journal,the population dynamics of microparasites and their invertebrate hosts,91660768,0,15,983,Philosophical Transactions of the Royal Society B,91660768
1984-01-01,2108273927,10.1098/RSTB.1984.0002,Journal,the limits to peat bog growth,91660768,0,35,902,Philosophical Transactions of the Royal Society B,91660768
1985-01-01,2115998792,10.1098/RSTB.1985.0010,Journal,actions and habits the development of behavioural autonomy,91660768,0,9,852,Philosophical Transactions of the Royal Society B,91660768


In [147]:
VID = 61274580
start_year = '1990'
end_year = '1999'

print("total number of papers:", len(model.paper_detailed_df[start_year:end_year][(model.paper_detailed_df[start_year:end_year].VenueID==VID)]))
model.paper_detailed_df[start_year:end_year][(model.paper_detailed_df[start_year:end_year].
                                              VenueID==VID)].sort_values('CitationCount', ascending=False).head(10)

total number of papers: 1


,PaperID,Doi,DocType,PaperTitle,JournalID,ConferenceSeriesID,ReferenceCount,CitationCount,OriginalVenue,VenueID
Year,,,,,,,,,,
1991-01-01,2498079382,0,Journal,comparative intergenerational stratification research,61274580,0,0,23,Annual Review of Sociology,61274580


In [34]:
focal_paper_list = model.paper_detailed_df[start_year:end_year][model.paper_detailed_df[start_year:end_year].OriginalVenue==venue_name].PaperID.to_list()

# Look up citation trail (random walks)

In [ ]:
focal_walks = []
vid = get_vid('English')
for walk in model.walks:
    if vid in walk:
        focal_walks.append(walk)

In [ ]:
focal_walks

In [ ]:
VID_list = [67764474.0, 67764474.0, 67764474.0, 86431830.0]
[model.MAG_venue_info_df.at[VID,'OriginalVenue'] for VID in VID_list]

# Loop up MAG reference records (ref.pkl)

In [65]:
start_year=1980
end_year=1989

In [31]:
model.load_ref_df(full_load=False, start_year, end_year)

2023-11-08 12:25:48 -- start loading 'ref_df'...
2023-11-08 12:25:49 -- finish.


In [63]:
for paper_ID in focal_paper_list:
    if len(model.target_ref_df[model.target_ref_df.PaperReferenceID == paper_ID]) !=0:
        display(model.target_ref_df[model.target_ref_df.PaperReferenceID == paper_ID])

,PaperReferenceID
PaperID,
1491928743,2315189172


,PaperReferenceID
PaperID,
2781934296,2319931332


,PaperReferenceID
PaperID,
13075566,2328347537
265484110,2328347537
1971602878,2328347537
1981137724,2328347537
1983500773,2328347537
1987161240,2328347537
1993474485,2328347537
2008987134,2328347537
2023797625,2328347537


,PaperReferenceID
PaperID,
2782136301,2313988925


,PaperReferenceID
PaperID,
2781947361,2315566305


,PaperReferenceID
PaperID,
2782425643,2333737838


,PaperReferenceID
PaperID,
2164128878,2014411277


In [85]:
model.paper_detailed_df[model.paper_detailed_df.PaperID==2328347537]

,PaperID,Doi,DocType,PaperTitle,JournalID,ConferenceSeriesID,ReferenceCount,CitationCount,OriginalVenue,VenueID
Year,,,,,,,,,,
1984-01-01,2328347537,10.1093/ENGLISH/33.147.263,Journal,a text book,86431830,0,0,177,English,86431830


In [95]:
model.paper_detailed_df[model.paper_detailed_df.PaperID==1987161240]

,PaperID,Doi,DocType,PaperTitle,JournalID,ConferenceSeriesID,ReferenceCount,CitationCount,OriginalVenue,VenueID
Year,,,,,,,,,,
1984-01-01,1987161240,10.1007/BF02166795,Journal,solvent effects on the37cl n γ 38cl reaction products of some solid chlorobenzene derivatives,63023845,0,6,4,Journal of Radioanalytical and Nuclear Chemistry,63023845
